# Project-2
## Tanvir Khan, Nicky Pant, Paul Pineda, James Ye, Fabienne Zumbuehl
### loadCSVFiles

In [1]:
import pandas as pd
#from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from sqlalchemy import create_engine
import psycopg2

In [2]:
# Creating connection with database
# StocksDataBase is database name for this project
connection_string = "postgres:postgres@localhost:5432/StocksDataBase"
engine = create_engine(f'postgresql://{connection_string}')

In [3]:
# Confirm tables
engine.table_names()

['company', 'price']

In [4]:
df_600104 = pd.read_csv("data/600104.SS.csv");
df_600104.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-07-01,25.780001,26.240000,25.780001,26.049999,24.764336,26499496
1,2019-07-02,26.000000,27.100000,25.820000,26.780001,25.458307,37020490
2,2019-07-03,26.809999,26.969999,26.600000,26.799999,25.477320,25985266
3,2019-07-04,26.850000,27.480000,26.799999,26.850000,25.524853,26854638
4,2019-07-05,26.850000,27.129999,26.459999,26.969999,25.638929,22252421


In [5]:
print(df_600104.dtypes)

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object


In [6]:
ticker = "600104.SS"
df_600104["ticker"] = ticker
df_600104.head()

,Date,Open,High,Low,Close,Adj Close,Volume,ticker
0,2019-07-01,25.780001,26.240000,25.780001,26.049999,24.764336,26499496,600104.SS
1,2019-07-02,26.000000,27.100000,25.820000,26.780001,25.458307,37020490,600104.SS
2,2019-07-03,26.809999,26.969999,26.600000,26.799999,25.477320,25985266,600104.SS
3,2019-07-04,26.850000,27.480000,26.799999,26.850000,25.524853,26854638,600104.SS
4,2019-07-05,26.850000,27.129999,26.459999,26.969999,25.638929,22252421,600104.SS


In [7]:
# price_data_df.rename(columns = {"SYMBOL":"symbol","NAME":"company", "PRICE":"price", "LOW":"low", "HIGH":"high", "PREVIOUS CLOSE":"previous_close"}, inplace=True)
df_600104.rename(columns = {"Date":"date", "Open":"open", "High":"high", "Low":"low", "Close":"close", "Adj Close":"adj_close", "Volume":"volume"}, inplace=True)
df_600104.head()

,date,open,high,low,close,adj_close,volume,ticker
0,2019-07-01,25.780001,26.240000,25.780001,26.049999,24.764336,26499496,600104.SS
1,2019-07-02,26.000000,27.100000,25.820000,26.780001,25.458307,37020490,600104.SS
2,2019-07-03,26.809999,26.969999,26.600000,26.799999,25.477320,25985266,600104.SS
3,2019-07-04,26.850000,27.480000,26.799999,26.850000,25.524853,26854638,600104.SS
4,2019-07-05,26.850000,27.129999,26.459999,26.969999,25.638929,22252421,600104.SS


In [8]:
df_600104 = df_600104[["ticker", "date", "open", "high", "low", "close", "adj_close", "volume"]]
df_600104.head()

,ticker,date,open,high,low,close,adj_close,volume
0,600104.SS,2019-07-01,25.780001,26.240000,25.780001,26.049999,24.764336,26499496
1,600104.SS,2019-07-02,26.000000,27.100000,25.820000,26.780001,25.458307,37020490
2,600104.SS,2019-07-03,26.809999,26.969999,26.600000,26.799999,25.477320,25985266
3,600104.SS,2019-07-04,26.850000,27.480000,26.799999,26.850000,25.524853,26854638
4,600104.SS,2019-07-05,26.850000,27.129999,26.459999,26.969999,25.638929,22252421


In [9]:
# insert ticker into company table if it doesn't exist yet, because it is the foreign key for the price table
connection = engine.connect()
result = connection.execute(f"select * from company where ticker = '{ticker}'")
rows = result.fetchall()
if len(rows) == 0:
    print(f"inserting {ticker} into company table")
    connection.execute(f"insert into company(ticker) values ('{ticker}')")
    result = connection.execute(f"select * from company where ticker = '{ticker}'")
    rows = result.fetchall()
for row in rows:
    print(row)
    


inserting 600104.SS into company table
('600104.SS', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
